In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
from IPython.display import clear_output
!pip3 install -U lazypredict
!pip3 install -U pandas  # Upgrading pandas

clear_output()  # 출력값 지우는 용도로 보임

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split

from lightgbm import LGBMClassifier
import lazypredict
from lazypredict.Supervised import LazyClassifier

import time
import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
train = pd.read_csv("../input/spaceship-titanic/train.csv")
test = pd.read_csv("../input/spaceship-titanic/test.csv")
submission = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")

RANDOM_STATE = 12
FOLDS = 5
STRATEGY = 'median'

## Column Descriptions:
- PassengerId: 각 승객의 고유 ID
- HomePlanet: 승객이 출발한 행성
- CryoSleep: 승객이 항해 기간 동안 애니메이션을 일시 중단하도록 선택했는지 여부를 나타냄
- Cabin: 승객이 머물고 있는 객실 번호입니다.
- Destination: 승객이 출발할 행성입니다.
- Age: 승객의 나이
- VIP: 승객이 항해 중 특별 VIP 서비스 비용을 지불했는지 여부.
- RoomService: 승객이 룸서비스에 대해 청구한 금액입니다.
- Name: 승객의 이름
- Transported: 승객이 다른 차원으로 운송되었는지 여부. 예측하려는 대상인 열


In [5]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.00,False,0.00,0.00,0.00,0.00,0.00,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.00,False,109.00,9.00,25.00,549.00,44.00,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.00,True,43.00,3576.00,0.00,6715.00,49.00,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.00,False,0.00,1283.00,371.00,3329.00,193.00,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.00,False,303.00,70.00,151.00,565.00,2.00,Willy Santantines,True


In [6]:
print(f'\033[94mNumber of rows in train data: {train.shape[0]}')
print(f'\033[94mNumber of columnsows in train data: {train.shape[1]}')
print(f'\033[94mNumber of values in train data: {train.count().sum()}')
print(f'\033[94mNumber of missing values in train data: {sum(train.isna().sum())}')
# 결측치 2324개

Number of rows in train data: 8693
Number of columnsows in train data: 14
Number of values in train data: 119378
Number of missing values in train data: 2324


- 열별 결측값

In [7]:
print(f'\033[94m')  # \033[94m : 출력 값에 색 적용
print(train.isna().sum().sort_values(ascending = False))


CryoSleep       217
ShoppingMall    208
VIP             203
HomePlanet      201
Name            200
Cabin           199
VRDeck          188
FoodCourt       183
Spa             183
Destination     182
RoomService     181
Age             179
PassengerId       0
Transported       0
dtype: int64


## 훈련데이터의 기본 통계

In [8]:
train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.00,8512.00,8510.00,8485.00,8510.00,8505.00
mean,28.83,224.69,458.08,173.73,311.14,304.85
std,14.49,666.72,1611.49,604.70,1136.71,1145.72
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,19.00,0.00,0.00,0.00,0.00,0.00
50%,27.00,0.00,0.00,0.00,0.00,0.00
75%,38.00,47.00,76.00,27.00,59.00,46.00
max,79.00,14327.00,29813.00,23492.00,22408.00,24133.00


## 테스트 데이터 확인

In [9]:
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.00,False,0.00,0.00,0.00,0.00,0.00,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.00,False,0.00,9.00,0.00,2823.00,0.00,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.00,False,0.00,0.00,0.00,0.00,0.00,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.00,False,0.00,6652.00,0.00,181.00,585.00,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.00,False,10.00,0.00,635.00,0.00,0.00,Brence Harperez


In [10]:
print(f'\033[94mNumber of rows in test data: {test.shape[0]}')
print(f'\033[94mNumber of columns in test data: {test.shape[1]}')
print(f'\033[94mNumber of values in train data: {test.count().sum()}')
print(f'\033[94mNo of rows with missing values in test data: {sum(test.isna().sum())}')

Number of rows in test data: 4277
Number of columns in test data: 13
Number of values in train data: 54484
No of rows with missing values in test data: 1117


## 열별 결측값

In [11]:
print(f'\033[94m')
print((test.isna().sum().sort_values(ascending = False)))


FoodCourt       106
Spa             101
Cabin           100
ShoppingMall     98
Name             94
CryoSleep        93
VIP              93
Destination      92
Age              91
HomePlanet       87
RoomService      82
VRDeck           80
PassengerId       0
dtype: int64


## 테스트 데이터의 기본 통계

In [12]:
test.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,4186.00,4195.00,4171.00,4179.00,4176.00,4197.00
mean,28.66,219.27,439.48,177.30,303.05,310.71
std,14.18,607.01,1527.66,560.82,1117.19,1246.99
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,19.00,0.00,0.00,0.00,0.00,0.00
50%,26.00,0.00,0.00,0.00,0.00,0.00
75%,37.00,53.00,78.00,33.00,50.00,36.00
max,79.00,11567.00,25273.00,8292.00,19844.00,22272.00


## 샘플 파일

In [13]:
submission.head()

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False


# EDA

## Overview of Data

In [14]:
train.drop(["PassengerId"], axis = 1, inplace = True)
test.drop(["PassengerId"], axis = 1, inplace = True)

TARGET = 'Transported'
FEATURES = [col for col in train.columns if col not in ['id', TARGET]]
RANDOM_STATE = 12

In [15]:
# 행과 열을 전환시키고 std(표준편차)기준으로 내림차순으로 정렬
train.iloc[:, :-1].describe().T.sort_values(by='std', ascending= False)\
.style.background_gradient(cmap='GnBu')\
.bar(subset= ["max"], color= '#BB0000')\
.bar(subset= ["mean"], color= 'green')

,count,mean,std,min,25%,50%,75%,max
FoodCourt,8510.00,458.08,1611.49,0.00,0.00,0.00,76.00,29813.00
VRDeck,8505.00,304.85,1145.72,0.00,0.00,0.00,46.00,24133.00
Spa,8510.00,311.14,1136.71,0.00,0.00,0.00,59.00,22408.00
RoomService,8512.00,224.69,666.72,0.00,0.00,0.00,47.00,14327.00
ShoppingMall,8485.00,173.73,604.70,0.00,0.00,0.00,27.00,23492.00
Age,8514.00,28.83,14.49,0.00,19.00,27.00,38.00,79.00


## Null값 분포

- 열별 Null값 분포

In [16]:
test_null = pd.DataFrame(test.isna().sum())
test_null = test_null.sort_values(by= 0, ascending = False)
train_null = pd.DataFrame(train.isna().sum())
train_null = train_null.sort_values(by= 0, ascending = False)[:-1]

fig = make_subplots(rows= 1,
                   cols= 2,
                   column_titles= ["Train Data", "Test Data"],
                   x_title= "Missing Values")

fig.add_trace(go.Bar(x= train_null.index,
                    y= train_null[0],
# fig.add_trace(go.Bar(x= train_null[0],  # column: 0
#                     y= train_null.index,  # 인덱스(행 기준)                     
                    orientation="v",
                    marker=dict(color= [n for n in range(12)],
                              line_color = 'rgb(0,0,0)',
                              line_width= 2,
                              coloraxis= "coloraxis")),
             1, 1)

fig.add_trace(go.Bar(x= test_null[0],
                    y= test_null.index,
                    orientation="h",
                    marker=dict(color= [n for n in range(12)],
                              line_color = 'rgb(0,0,0)',
                              line_width= 2,
                              coloraxis= "coloraxis")),
             1, 2)

fig.update_layout(showlegend= False, title_text= 'Column wise Null Value Distribution', title_x= 0.5)
# fig.update_layout(showlegend=False)

- 행별 Null값 분포

In [17]:
missing_train_row = train.isna().sum(axis= 1)  # null값을 더함.  axis= 1: 열 기준
# print(missing_train_row)
missing_train_row = pd.DataFrame(missing_train_row.value_counts()/train.shape[0]).reset_index()
# print(missing_train_row)
missing_test_row = test.isna().sum(axis= 1)
missing_test_row = pd.DataFrame(missing_test_row.value_counts()/test.shape[0]).reset_index()

missing_train_row.columns = ['no', 'count']  # 컬럼명 변경
missing_test_row.columns = ['no', 'count']
# print(missing_train_row)

missing_train_row['count']= missing_train_row['count']* 100  # 퍼센트로 표시
missing_test_row['count']= missing_test_row['count']* 100
# print(missing_train_row)

fig= make_subplots(rows= 1,
                  cols= 2,
                  column_titles = ['Train Data', 'Test Data'],
                  x_title= 'Missing Values')

fig.add_trace(go.Bar(x= missing_train_row['no'],
                    y= missing_train_row['count'],
                    marker = dict(color= [n for n in range(4)],
                                 line_color= 'rgb(0,0,0)',
                                 line_width= 3,
                                 coloraxis= 'coloraxis')),
              1, 1)

fig.add_trace(go.Bar(x= missing_test_row['no'],
                    y= missing_test_row['count'],
                    marker = dict(color= [n for n in range(4)],
                                 line_color= 'rgb(0,0,0)',
                                 line_width= 3,
                                 coloraxis= 'coloraxis')),
              1, 2)

fig.update_layout(showlegend= False, title_text= 'Row wise Null Value Distribution', title_x= 0.5)

- Null 값 분포의 관찰
    - 12개의 특성중 6개의 특성은 연속형, 2개의 특성은 텍스트 데이터, 4개의 특성은 범주형
    - HomePlanet 및 Destination에는 3개의 서로 다른 고유 값이 있음
    - CryoSleep 및 VIP는 범주형 특성입니다.

# 정리
- clear_output(): 출력값을 지우는 함수. ex) 라이브러리 설치 후 사용
- 리스트[:-1]: 이 리스트의 마지막번째는 제외
- 
- Pandas라이브러리
    - T: 로우와 컬럼을 바꿈
    - sort_values(): 컬럼을 기준으로 정렬 시키는 함수
        - 매개변수 by: 정렬의 기준이 되는 컬럼을 지정
        - 매개변수 asceding: 오름차순(True), 내림차순(False). [default: True]
    - sort_index(): 인덱스(로우)를 기준으로 정렬시키는 함수
    - isna(): 결측값으로 True / False로 구분
- Plotly 패키지
    - make_subplots(): fig(틀)를 잡는 함수.
        - 매개변수 rows: 행의 개수를 지정
        - 매개변수 cols: 열의 개수를 지정
        - 매개변수 x_title: fig(틀)의 x축의 제목을 지정
    - update_layout(): 그래프를 보여주는 함수.
        - 매개변수 showlegend: 각각의 그래프를 보여주고 숨기는 기능 [default: True](기능 사용함)
        - ***매개변수 title_x: 알 수 없음***
    - go.bar(): 막대 그래프를 그리는 함수
        - 매개변수 orientation: 위로 향하는("v")그래프를 그릴것인지, 옆으로 향하는("h") 그래프를 그릴것인지 지정. [default: "v"]